# SentencePiece Python module
* SentencePiece: 내부 단어 분리를 위한 유용한 패키지
*참고: https://colab.research.google.com/github/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb#scrollTo=T9BDzLVkUFT4
https://wikidocs.net/86657

## SentencePiece 학습 코드
* 참고: http://tensorboy.com/bpe-sentencepiece

* 입력: corpus(말뭉치), prefix(생성할 파일 이름), vocab_size(vocabulary 개수)

In [1]:
def train_sentencepiece(corpus, prefix, vocab_size=32000):
    """
    sentencepiece를 이용해 vocab 학습
    :param corpus: 학습할 말뭉치
    :param prefix: 저장할 vocab 이름
    :param vocab_size: vocab 개수
    """
    spm.SentencePieceTrainer.train(
        f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +  # 7은 특수문자 개수
        " --model_type=unigram" +
        " --max_sentence_length=999999" +  # 문장 최대 길이
        " --pad_id=0 --pad_piece=[PAD]" +  # pad token 및 id 지정
        " --unk_id=1 --unk_piece=[UNK]" +  # unknown token 및 id 지정
        " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence token 및 id 지정
        " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence token 및 id 지정
        " --user_defined_symbols=[SEP],[CLS],[MASK]" +  # 기타 추가 토큰 SEP: 4, CLS: 5, MASK: 6
        " --input_sentence_size=100000" +  # 말뭉치에서 셈플링해서 학습
        " --shuffle_input_sentence=true")  # 셈플링한 말뭉치 shuffle

함수 시행 시, 다음 두 파일 생성
* '<'prefix'>.model': Sentencepiece가 학습한 내용을 읽어 들이기 위한 파일.
* '<'prefix'>.vocab': text 형식으로 내용을 확인할 수 있는 파일

## Install and data preparation

training data: botchan.txt, English-translated

In [2]:
# !pip install sentencepiece
!wget https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt

--2022-09-08 10:30:01--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘botchan.txt’

botchan.txt         100%[===================>] 272.25K  --.-KB/s    in 0.05s   

2022-09-08 10:30:01 (5.55 MB/s) - ‘botchan.txt’ saved [278779/278779]



In [3]:
import warnings
warnings.filterwarnings(action='ignore')

## Basic end-to-end example

### train 
Vocab 만들기 |
센텐스피스로 단어 집합과 각 단어에 고유한 정수를 부여하기
* input : 학습시킬 파일
* model_prefix : 만들어질 모델 이름
* vocab_size : 단어 집합의 크기
------
* model_type : 사용할 모델 (unigram(default), bpe, char, word)
* max_sentence_length: 문장의 최대 길이
* pad_id, pad_piece: pad token id, 값
* unk_id, unk_piece: unknown token id, 값
* bos_id, bos_piece: begin of sentence token id, 값
* eos_id, eos_piece: end of sequence token id, 값
* user_defined_symbols: 사용자 정의 토큰

* 토크나이저의 학습: 학습 문장들을 토대로 문장을 쪼개는 방식을 학습하는 것
* 텍스트 파일 -> 학습 -> model, vocab 파일
  * 문장들이 나열된 텍스트 파일을 넣고 학습시키면 model, vocab 파일이 만들어지는데
    * model이 실제 토크나이징을 하고,
    * vocab은 토크나이징 할 때 참조하는 단어집합
    * 해당 예시의 경우, botchan.txt 파일을 읽은 후, m.model과 m.vocab 파일을 생성하게 된다

In [4]:
import sentencepiece as spm

# train sentencepiece model from `botchan.txt` and makes `m.model` and `m.vocab`
# `m.vocab` is just a reference. not used in the segmentation.
spm.SentencePieceTrainer.train('--input=botchan.txt --model_prefix=m --vocab_size=2000')

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=botchan.txt --model_prefix=m --vocab_size=2000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: botchan.txt
  input_format: 
  model_prefix: m
  model_type: UNIGRAM
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:

### model load
* 토크나이징 진행에 앞서 생성한 model을 불러온다
* model 파일을 로드하여 단어 시퀀스를 정수 시퀀스로 바꾸는 인코딩 작업이나 반대로 변환하는 디코딩 작업을 할 수 있다
* 로딩이 잘 된 경우 'True'를 반환

In [5]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

### tokenize(tokenize 수행하기)
#### encode
* encode : 문장으로부터 인자값에 따라서 정수 시퀀스 또는 서브워드 시퀀스로 변환 가능합니다.
1. encode_as_pieces : string으로 tokenize, 문장을 입력하면 서브 워드 시퀀스로 변환합니다.
2. encode_as_ids : ids으로 tokenize, 문장을 입력하면 정수 시퀀스로 변환합니다.

#### decode
* DecodeIds : 정수 시퀀스로부터 문장으로 변환합니다.
* DecodePieces : 서브워드 시퀀스로부터 문장으로 변환합니다.

In [6]:
# encode: text => id
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

# decode: id => text
print(sp.decode_pieces(['▁This', '▁is', '▁a', '▁t', 'est']))
print(sp.decode_ids([209, 31, 9, 375, 586]))

['▁This', '▁is', '▁a', '▁t', 'est']
[209, 31, 9, 375, 586]
This is a test
This is a test


결과 해석: {단어: id} 대응값을 사전으로 저장한 것이 아까 생성된 m.vocab 파일


### 그 외 기능들
GetPieceSize() : 단어 집합의 크기를 확인합니다.

In [7]:
# returns vocab size
print(sp.get_piece_size())

2000


1. idToPiece : 정수로부터 맵핑되는 서브 워드로 변환합니다.
2. PieceToId : 서브워드로부터 맵핑되는 정수로 변환합니다.

In [8]:
# id <=> piece conversion
print(sp.id_to_piece(209))
print(sp.piece_to_id('▁This'))

# returns 0 for unknown tokens (we can change the id for UNK)
print(sp.piece_to_id('__MUST_BE_UNKNOWN__')) # 알수 없는 토큰의 경우 0을 반환

▁This
209
0


In [9]:
# <unk>, <s>, </s> are defined by default. Their ids are (0, 1, 2)
# <s> and </s> are defined as 'control' symbol.
for id in range(3):
    print(sp.id_to_piece(id), sp.is_control(id))

<unk> False
<s> True
</s> True


## Changing the vocab id and surface representation of UNK/BOS/EOS/PAD symbols
By default, UNK/BOS/EOS/PAD tokens and their ids are defined as follows:

|token|UNK|BOS|EOS|PAD| ---|--- |surface|<unk>|<s>|</s>|<pad>| |id|0|1|2|undefined (-1)|

We can change these mappings with --{unk|bos|eos|pad}_id and --{unk|bos|eos|pad}_piece flags.

센텐스피스로 단어 집합과 각 단어에 고유한 정수를 부여하기
* input : 학습시킬 파일
* vocab_size : 단어 집합의 크기
* model_prefix : 만들어질 모델 이름
* pad_id, pad_piece: pad token id, 값
* unk_id, unk_piece: unknown token id, 값
* bos_id, bos_piece: begin of sentence token id, 값
* eos_id, eos_piece: end of sequence token id, 값

------
* model_type : 사용할 모델 (unigram(default), bpe, char, word)
* max_sentence_length: 문장의 최대 길이
* user_defined_symbols: 사용자 정의 토큰 # 

In [11]:
spm.SentencePieceTrainer.train('--input=botchan.txt --vocab_size=2000 --model_prefix=m --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3 --pad_piece=[PAD] --unk_piece=[UNK] --bos_piece=[BOS] --eos_piece=[EOS]')
sp = spm.SentencePieceProcessor()
sp.load('m.model')


for id in range(4):
    print(sp.id_to_piece(id), sp.is_control(id))

[PAD] True
[UNK] False
[BOS] True
[EOS] True


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=botchan.txt --vocab_size=2000 --model_prefix=m --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3 --pad_piece=[PAD] --unk_piece=[UNK] --bos_piece=[BOS] --eos_piece=[EOS]
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: botchan.txt
  input_format: 
  model_prefix: m
  model_type: UNIGRAM
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 1
  bos_id

## BPE (Byte pair encoding) model
Sentencepiece는 --model_type=bpe flag로 지정해 subword segmentation을 위한 BPE (byte-pair-encoding)를 지원한다. 
* unigram과의 결과 비교

In [12]:
spm.SentencePieceTrainer.train('--input=botchan.txt --model_prefix=m_bpe --vocab_size=2000 --model_type=bpe')
sp_bpe = spm.SentencePieceProcessor()
sp_bpe.load('m_bpe.model')

print('*** BPE ***')
print(sp_bpe.encode_as_pieces('thisisatesthelloworld'))
print(sp_bpe.nbest_encode_as_pieces('hello world', 5))  # returns an empty list.

*** BPE ***
['▁this', 'is', 'at', 'est', 'he', 'llow', 'or', 'ld']
[]


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=botchan.txt --model_prefix=m_bpe --vocab_size=2000 --model_type=bpe
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: botchan.txt
  input_format: 
  model_prefix: m_bpe
  model_type: BPE
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece:

In [13]:
spm.SentencePieceTrainer.train('--input=botchan.txt --model_prefix=m_unigram --vocab_size=2000 --model_type=unigram')
sp_unigram = spm.SentencePieceProcessor()
sp_unigram.load('m_unigram.model')

print('*** Unigram ***')
print(sp_unigram.encode_as_pieces('thisisatesthelloworld'))
print(sp_unigram.nbest_encode_as_pieces('thisisatesthelloworld', 5))

*** Unigram ***
['▁this', 'is', 'ate', 's', 'the', 'llow', 'or', 'l', 'd']
[['▁this', 'is', 'ate', 's', 'the', 'llow', 'or', 'l', 'd'], ['▁this', 'i', 's', 'ate', 's', 'the', 'llow', 'or', 'l', 'd'], ['▁this', 'is', 'ate', 'st', 'he', 'llow', 'or', 'l', 'd'], ['▁this', 'is', 'at', 'es', 'the', 'llow', 'or', 'l', 'd'], ['▁this', 'is', 'at', 'est', 'he', 'llow', 'or', 'l', 'd']]


## Character and word model
Sentencepiece를 통해
character는 --model_type=char로,
word는 --model_type=character로 segmentation이 가능하다

In word segmentation에서 sentencepiece는 tokens를 그저 whitespaces로 segment하기 때문에,input text는 pre-tokenized되어야 한다. 

아래 결과 두 개 비교!

In [14]:
spm.SentencePieceTrainer.train('--input=botchan.txt --model_prefix=m_char --model_type=char --vocab_size=400')

sp_char = spm.SentencePieceProcessor()
sp_char.load('m_char.model')

print(sp_char.encode_as_pieces('this is a test.'))
print(sp_char.encode_as_ids('this is a test.'))

['▁', 't', 'h', 'i', 's', '▁', 'i', 's', '▁', 'a', '▁', 't', 'e', 's', 't', '.']
[3, 5, 10, 9, 11, 3, 9, 11, 3, 7, 3, 5, 4, 11, 5, 23]


In [15]:
spm.SentencePieceTrainer.train('--input=botchan.txt --model_prefix=m_word --model_type=word --vocab_size=2000')

sp_word = spm.SentencePieceProcessor()
sp_word.load('m_word.model')

print(sp_word.encode_as_pieces('this is a test.'))  # '.' will not be one token.
print(sp_word.encode_as_ids('this is a test.'))

['▁this', '▁is', '▁a', '▁test.']
[31, 17, 8, 0]


## Text normalization
Sentencepiece 는 일반적인 사전 정의된 normalization rules를 제공한다. normalizer를 --normaliation_rule_name=<NAME> flag를 이용해 변경 가능하다.

* nmt_nfkc: NFKC normalization with some additional normalization around spaces. (default)
* nfkc: original: NFKC normalization.
* nmt_nfkc_cf: nmt_nfkc + Unicode case folding (mostly lower casing)
* nfkc_cf: nfkc + Unicode case folding.
* identity: no normalization

In [16]:
# NFKC normalization and lower casing.
spm.SentencePieceTrainer.train('--input=botchan.txt --model_prefix=m --vocab_size=2000 --normalization_rule_name=nfkc_cf')

sp = spm.SentencePieceProcessor()
sp.load('m.model')
print(sp.encode_as_pieces('ＨＥＬＬＯ　ＷＯＲＬＤ.'))  # lower casing and normalization

['▁', 'hello', '▁world', '.']


The normalization is performed with user-defined string-to-string mappings and leftmost longest matching. We can also define the custom normalization rules as TSV file. The TSV files for pre-defined normalziation rules can be found in the data directory (sample). The normalization rule is compiled into FST and embedded in the model file. We don't need to specify the normalization configuration in the segmentation phase.

Here's the example of custom normalization. The TSV file is fed with --normalization_rule_tsv=<FILE> flag.

In [17]:
def tocode(s):
    out = []
    for c in s:
        out.append(str(hex(ord(c))).replace('0x', 'U+'))
    return ' '.join(out)


# TSV format:  source Unicode code points <tab> target code points
# normalize "don't => do not,  I'm => I am"
with open('normalization_rule.tsv', 'w') as f:
  f.write(tocode("I'm") + '\t' + tocode("I am") + '\n')
  f.write(tocode("don't") + '\t' + tocode("do not") + '\n')

print(open('normalization_rule.tsv', 'r').read())

spm.SentencePieceTrainer.train('--input=botchan.txt --model_prefix=m --vocab_size=2000 --normalization_rule_tsv=normalization_rule.tsv')

sp = spm.SentencePieceProcessor()
# m.model embeds the normalization rule compiled into an FST.
sp.load('m.model')
print(sp.encode_as_pieces("I'm busy"))  # normalzied to `I am busy'
print(sp.encode_as_pieces("I don't know it."))  # normalized to 'I do not know it.'

U+49 U+27 U+6d	U+49 U+20 U+61 U+6d
U+64 U+6f U+6e U+27 U+74	U+64 U+6f U+20 U+6e U+6f U+74

['▁I', '▁am', '▁bu', 's', 'y']
['▁I', '▁do', '▁not', '▁know', '▁it', '.']


## Randomizing training data
* Sentencepiece는 모든 training data를 model을 훈련하는데 load한다.
* 하지만 training data가 크면, training time과 memory usage가 증가한다.
* 만일 --input_sentence_size=<SIZE> 가 명시되어 있다면, Sentencepiece 랜덤하게 전체 훈련 데이터에서 <SIZE> lines를 샘플한다.
* --shuffle_input_sentence=false는 랜덤 셔플을 불가하게 하고 first <SIZE> lines를 취하게 한다

In [18]:
spm.SentencePieceTrainer.train('--input=botchan.txt --model_prefix=m --vocab_size=2000 --input_sentence_size=1000')

sp = spm.SentencePieceProcessor()
sp.load('m.model')

sp.encode_as_pieces('this is a test.')

['▁this', '▁is', '▁a', '▁t', 'est', '.']